## 2024 06/14 AutoML test

*Last Updated*: 2024-06-14

### Authors
* Nicole Tin (...)


### Overview
This Jupyter notebook is intended to demonstrate

* A...


### Key Results

- E...

In [ ]:
# --- Imports

# External packages
import hub
from pycaret import regression
import os
import cv2
from dermaml import data
from dermaml import features
# import PIL
# from PIL import Image
# import numpy as np
# import mlflow
# import matplotlib.pyplot as plt
import pandas as pd
# import skimage


In [ ]:
# Dataset
dataset_name = "11khands"

# AutoML
experiment_name = "11khands-automl-sample-test_NT"
num_best_models = 5
random_seed = 42

In [ ]:
# Images
ds = hub.load('hub://activeloop/11k-hands')
images = ds.images